In [1]:
import astro_ghost
astro_ghost.__version__

'0.2.3'

In [2]:
import os
import sys
import time
import astro_ghost
# from astro_ghost.PS1QueryFunctions import getAllPostageStamps
# from astro_ghost.TNSQueryFunctions import getTNSSpectra
# from astro_ghost.NEDQueryFunctions import getNEDSpectra
from astro_ghost.ghostHelperFunctions import getTransientHosts, getGHOST
from astropy.coordinates import SkyCoord
from astropy import units as u
import pandas as pd
from datetime import datetime
import numpy as np

import tempfile

In [3]:
dataset_bank = pd.read_csv(f'../loci_dbs/alerce_cut/dataset_bank_and_tns_df_9420objects.csv.gz', 
                           compression='gzip', index_col='ztf_object_id')
dataset_bank

/Users/patrickaleo/miniconda3/envs/laiss/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (125,133) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,locus_id,ra,dec,tags,catalogs,ztf_ssnamenr,num_alerts,num_mag_values,oldest_alert_id,...,vpdf_extreme_version,vpdf_extreme_faint,vpdf_extreme_bright,locus_gal_l,locus_gal_b,stamp_cls,has_tns,tns_cls,spec_z,report_group
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF17aaadars,858089,ANT2020setxa,74.008152,4.688007,"['dwarf_nova_outburst', 'high_snr', 'high_ampl...",['bright_guide_star_cat'],NaN,176,50,ztf_candidate:726272534215015034,...,NaN,NaN,NaN,194.434930,-23.001128,SNIa,No_TNS,None,-99.0,None
ZTF17aaadvfh,845168,ANT2020jk2po,10.123141,40.602475,"['in_m31', 'lc_feature_extractor', 'extragalac...","['RC3', '2mass_xsc', 'gaia_dr2']",NaN,345,25,ztf_candidate:1007342854215010039,...,NaN,NaN,NaN,120.687977,-22.221579,SLSN,No_TNS,None,-99.0,None
ZTF17aaahrni,896108,ANT2020aevgf7q,80.423285,-4.186247,"['high_amplitude_transient_candidate', 'nova_t...",[],NaN,63,16,ztf_candidate:1398444022315015011,...,NaN,NaN,NaN,206.213250,-21.841968,SNIa,Yes_TNS,None,-99.0,GaiaAlerts
ZTF17aaaivqs,985055,ANT2020jq7w6,127.370917,10.773342,"['high_amplitude_variable_star_candidate', 'lc...","['asassn_variable_catalog_v2_20190802', 'allwi...",NaN,358,292,ztf_candidate:671461312615010000,...,NaN,NaN,NaN,214.180505,26.612885,SLSN,No_TNS,None,-99.0,None
ZTF17aaaizmo,998136,ANT2020pg5q4,132.155248,10.782338,"['matheson_extreme_vpdf', 'lc_feature_extracto...","['2mass_psc', 'bright_guide_star_cat', 'gaia_d...",NaN,289,97,ztf_candidate:680499561915015002,...,0.1.0,pct_99.99999_outlier,NaN,216.450964,30.870773,SNII,No_TNS,None,-99.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF22aacjbmo,909153,ANT2020euljm,205.509960,-19.933047,['lc_feature_extractor'],"['allwise', 'bright_guide_star_cat', '2mass_ps...",NaN,132,60,ztf_candidate:738499311115015008,...,NaN,NaN,NaN,318.854533,41.368904,SLSN,No_TNS,None,-99.0,None
ZTF22aackync,274345,ANT2020aeh34ei,302.025743,-2.811300,['lc_feature_extractor'],"['bright_guide_star_cat', '2mass_psc', 'allwis...",NaN,273,88,ztf_candidate:1357212311615015001,...,NaN,NaN,NaN,39.334932,-18.312848,SNII,No_TNS,None,-99.0,None
ZTF22aacreno,281787,ANT2020edvwq,146.940490,-21.929953,['lc_feature_extractor'],"['milliquas', 'bright_guide_star_cat', 'gaia_d...",NaN,129,95,ztf_candidate:828188395815015007,...,NaN,NaN,NaN,256.172074,23.831250,SLSN,No_TNS,None,-99.0,None


In [ ]:
%%time

# Throw RA/DEC into ghost with just DLR method, gentle starcut
# Sets environ var to find ghost.csv
os.environ['GHOST_PATH'] = './host_info'
# Then don't use getGHOST(real=True, verbose=verbose)
getGHOST(real=True,verbose=False)

start_idx_l = list(np.arange(400, len(dataset_bank), 100)) 
end_idx_l = list(np.arange(500, len(dataset_bank)+100, 100))

for start, end in zip(start_idx_l, end_idx_l):
    start_idx, end_idx = start, end
    print("\nSTART:", start, "END:", end)

    dataset_bank_mini = dataset_bank.iloc[start_idx:end_idx]
    #print(dataset_bank_mini)

    snCoord = [SkyCoord(ra*u.deg, dec*u.deg, frame='icrs') for ra, dec in zip(dataset_bank_mini.ra, dataset_bank_mini.dec)]
    with tempfile.TemporaryDirectory() as tmp:
#        try: 
        hosts = getTransientHosts(snName=list(dataset_bank_mini.index), snCoord=snCoord, verbose=0, starcut='gentle', ascentMatch=False, savepath=tmp)
        # Save
        hosts.to_csv(f'../loci_dbs/alerce_cut/dataset_bank_hosts_start_idx-end_idx={start_idx}-{end_idx}.csv.gz', compression='gzip', index=False)
        print("Saved!\n#############")
#        except: 
        #print(f"Throws 500 error. Continue!")
        #continue
        

GHOST database already exists in the install path!

START: 300 END: 400
Finding northern sources with Pan-starrs...
Processed 10 of 82 lines!
Processed 20 of 82 lines!
Processed 30 of 82 lines!
Processed 40 of 82 lines!
Processed 50 of 82 lines!
Processed 60 of 82 lines!
Processed 70 of 82 lines!
Processed 80 of 82 lines!


/Users/patrickaleo/miniconda3/envs/laiss/lib/python3.8/site-packages/astro_ghost/stellarLocus.py:64: RuntimeWarning: All-NaN slice encountered
  df.loc[i,"7DCD"] = np.nanmin(np.array(temp_7DCD_1val))


Found matches for 43.9% of events.
Saved!
#############

START: 400 END: 500
Finding northern sources with Pan-starrs...
Processed 10 of 83 lines!
Processed 20 of 83 lines!
Processed 30 of 83 lines!
Processed 40 of 83 lines!
Processed 50 of 83 lines!
Processed 60 of 83 lines!
Processed 70 of 83 lines!
Processed 80 of 83 lines!
